In [1]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r /kaggle/working/multimodal-eq-sizing/requirements.txt

Cloning into '/kaggle/working/multimodal-eq-sizing'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 113 (delta 39), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (113/113), 23.57 KiB | 3.93 MiB/s, done.
Resolving deltas: 100% (39/39), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.9/259.9 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


In [2]:
import sys
import pathlib
import pandas as pd
from datetime import datetime

In [3]:
repo_root = pathlib.Path("/kaggle/working/multimodal-eq-sizing")
sys.path.append(str(repo_root))

In [4]:
from src.data.loaders import get_return_data, get_single_ticker_history

# Compute excess return

In [5]:
def get_date_range(df: pd.DataFrame) -> tuple:
    grouped_by_date = df.groupby(["ticker"]).agg(['min', 'max', 'count'])["Date"]
    start = grouped_by_date["min"].min()
    end = grouped_by_date["max"].max()
    return start, end

In [6]:
def compute_excess_return():
    df_ticker_return = get_return_data("/kaggle/input/news-trading/return_data.csv")
    start, end = get_date_range(df_ticker_return)
    df_base_ticker_return = get_single_ticker_history("SPY", start, end)
    
    return_formula = "(Close / Open) - 1"
    df_ticker_return["o2c_return"] = df_ticker_return.eval(return_formula)
    df_base_ticker_return["spy_o2c_return"] = df_base_ticker_return.eval(return_formula)
    df_base_ticker_return = df_base_ticker_return[["Date", "spy_o2c_return"]]

    excess_return_formula = "o2c_return - spy_o2c_return"
    df_with_target = pd.merge(df_ticker_return, df_base_ticker_return, on="Date", how="left")
    df_with_target["excess_return"] = df_with_target.eval(excess_return_formula)
    df_with_target = df_with_target.drop(['o2c_return', 'spy_o2c_return'], axis=1)
    
    return df_with_target

In [7]:
df_with_target = compute_excess_return()
df_with_target.to_csv('return_data_with_target.csv', index=False)

In [8]:
!rm -rf /kaggle/working/multimodal-eq-sizing